### 1. Importing Libraries
Barcoding system is a cheap and reliable way for tagging the products. Barcode detection process is needed for an invetory system to detect the barcodes of the products and for the billing system of the inventory concerned. 


In [2]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import time

### 2. Load the image and convert it to grayscale


In [3]:
image= cv2.imread("barcode_04.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

###  3. Compute the Scharr gradient magnitude representation of the images in both the x and y direction

In [4]:
ddepth = cv2.cv.CV_32F if imutils.is_cv2() else cv2.CV_32F
gradX = cv2.Sobel(gray, ddepth=ddepth, dx=1, dy=0, ksize=-1)
gradY = cv2.Sobel(gray, ddepth=ddepth, dx=0, dy=1, ksize=-1)
print(gradX)
print(gradY)

[[  -0.   40.  -56. ...   34.  -46.   -0.]
 [  -0.   21.  -78. ...   43.  -43.   -0.]
 [  -0.   15.  -49. ...    3.  -97.   -0.]
 ...
 [  -0.   -2.   -9. ...  -77.  -50.   -0.]
 [  -0.  -82.  -36. ... -103.  -31.   -0.]
 [  -0.  -92.  -38. ... -102.  -56.   -0.]]
[[   0.    0.    0. ...    0.    0.    0.]
 [  -2.  -17.  -18. ...   11.  -17.  -22.]
 [  -6.  -57.  -29. ...  -85. -155. -176.]
 ...
 [  18.  -32.  -91. ... -169. -144. -104.]
 [  62.   32.   -4. ...   49.   25.   16.]
 [   0.    0.    0. ...    0.    0.    0.]]


### 4. Subtract the y-gradient from the x-gradient

In [5]:
gradient = cv2.subtract(gradX, gradY)
gradient = cv2.convertScaleAbs(gradient)

### 5. Blur and threshold the image

In [6]:
blurred = cv2.blur(gradient, (9, 9))
(_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)

### 6. Construct a closing kernel and apply it to the thresholded image

In [7]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

### 7. Perform a series of erosions and dilations

In [8]:
closed = cv2.erode(closed, None, iterations = 4)
closed = cv2.dilate(closed, None, iterations = 4)

### 8. Find the contours in the thresholded image, sorting the contours by their area, keeping only the largest one

In [9]:
cnts = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]

### 9. Compute the rotated bounding box of the largest contour

In [12]:
rect = cv2.minAreaRect(c)
box = cv2.cv.BoxPoints(rect) if imutils.is_cv2() else cv2.boxPoints(rect)
box = np.int0(box)

### 10. Draw a bounding box around the detected barcode and display the image

In [ ]:
cv2.drawContours(image, [box], -1, (0, 255, 0), 3)
cv2.imshow("image", image)
cv2.waitKey(0)